In [1]:
import tensorflow as tf

In [1]:
import numpy as np

In [109]:
import qiskit
from scipy.linalg import expm


In [6]:
weights = np.random.normal(size=(2**(4*v), 1))


In [112]:
weights.shape

qubits = 16
steps = int(np.log2(qubits))

In [113]:
gates_per_step = []
for i in range(1,steps+1):
    gates_per_step.append(qubits//(2**i))
gates_per_step 

[8, 4, 2, 1]

In [55]:
params = 2**(4*v)*sum(gates_per_step)

In [107]:
from qiskit.quantum_info.operators.predicates import (is_hermitian_matrix,
                                                      is_unitary_matrix)


In [90]:
weights = np.random.normal(size=(int(sum(gates_per_step)), 2**(4*v)))
weights = weights[0]
weights

array([-0.61653241, -0.25805993, -0.96188335,  1.55362825,  0.59733385,
       -0.82732927, -1.35153459,  0.08732493, -1.521356  , -0.33762162,
        0.32980064, -0.06917155, -0.08559301,  0.75805073,  2.01477143,
       -0.49177805])

In [114]:
def unitary_from_hermitian(hermitian):
    """Generates a unitary matrix from a hermitian matrix.
        The formula is U = e^(i*H).

    Args:
        hermitian: A hermitian matrix.

    Returns:
        unitary: The resulting unitarian matrix.

    Raises:
        AssertionError: If the resulting matrix is not unitarian.
    """
    unitary = np.matrix(expm(1j * hermitian))
    assert is_unitary_matrix(unitary)
    return unitary


def hermitian_from_weights(weights, dimension):
    """Generates a  complex hermitian matrix from a set of weights.
        The hermitian is constructed by an upper triangle matrix which then is
        added to its transpose. The first dimension weights are used for the real
        diagonal values, the next values are used for the real parts of the upper
        triangle the rest for the imaginarie parts.

    Args:
        weights: List of weights.
        dimension: size of the matrix.

    Returns:
        hermitian: The resulting hermitian matrix.

    Raises:
        AssertionError: If the resulting matrix is not hermitian.
    """
    diagonals = weights[:dimension]
    dim = ((dimension**2 - dimension) // 2) + dimension
    reals = weights[dimension:dim]
    print(reals)
    imaginaries = weights[dim:]
    print(imaginaries)
    assert reals.shape == imaginaries.shape
    diag = np.matrix(np.diag(diagonals))
    hermitian = np.matrix(np.zeros((dimension,dimension), dtype=complex))
    hermitian[np.triu_indices(dimension, 1)] = np.array(
        [complex(a, b) for a, b in zip(reals, imaginaries)])
    hermitian = hermitian + hermitian.H + diag  # tril and triu don't use the same ordering!
    assert is_hermitian_matrix(hermitian)
    return hermitian


def unitaries_from_weights(weights, dimension=4):
    """Wrapper function to generate unitary matricies from weight list.

    Args:
        weights: The weights to transform.

    Returns:
        unitaries: The resulting list of unitary matrices.
    """
    unitaries = []
    for weight in weights:
        unitaries.append(
            unitary_from_hermitian(hermitian_from_weights(weight, dimension)))
    return unitaries



10

[[ 0.        +0.j          0.59733385+0.32980064j -0.82732927-0.06917155j
  -1.35153459-0.08559301j]
 [ 0.        +0.j          0.        +0.j          0.08732493+0.75805073j
  -1.521356  +2.01477143j]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
  -0.33762162-0.49177805j]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]]


NameError: name 'is_hermitian_matrix' is not defined

In [115]:
base_circuit = qiskit.QuantumCircuit(dimension, 1)

weights = np.random.normal(size=(int(sum(gates_per_step)), 2**(4*v)))

unitaries = unitaries_from_weights(weights) # Theoretically we have 15 unitaries


index = 0
for i in range(len(gates_per_step)):
    for j in range(gates_per_step[i]):
        qubits = []
        lower = step_size * j + 2**i - 1
        upper = step_size * j + step_size
        qubits.append(base_circuit.qubits[lower])
        qubits.append(base_circuit.qubits[upper - 1])
        base_circuit.unitary(unitaries[index], qubits, f'U({i},{j})')
        index += 1
base_circuit.measure([dimension - 1], [0])

[-0.80188114 -1.228659    0.43548494  0.49197316  0.76509507 -2.01568188]
[-0.3232818   0.08423062  2.8596082  -1.44204407  0.44144078 -1.01910207]
[-0.34502721  0.0475294   1.43054358 -0.48934324 -0.85134716 -1.58723763]
[ 0.38832741  0.00175392  1.06930924 -1.16631009  0.02174016 -0.81206183]
[ 0.44041085 -0.70407063  0.63468108 -0.05686144  0.56224912 -0.194959  ]
[-9.56047639e-01  1.32983265e-03  7.99268037e-01  1.96191179e+00
 -1.35453022e+00 -4.43547835e-01]
[ 0.08527857 -0.00575754  0.34629393 -0.480302    0.18029241 -0.02466638]
[ 2.36894044  0.48333911 -1.57509119  1.41114378 -0.48551818 -0.02268053]
[ 2.03968572 -0.4183232   1.62294743  1.01766239  0.0874474   0.36151814]
[-0.99307511  1.18630735 -0.70990139 -0.09389978 -2.13498692  1.41036251]
[-0.90680797 -0.21753844 -1.35487343 -0.25780077 -0.72196113  0.05805215]
[-1.22000465 -0.59113708 -0.38064271 -0.88801338 -0.06873617  0.25607384]
[-2.62945989 -0.62901938 -0.94682437 -0.94091447 -0.62587322  1.37032986]
[ 0.93635319 

NameError: name 'step_size' is not defined

In [126]:
a = [8, 4, 2, 1]
for i in range(4):
    step = 2**(i+1)
    print(f'step = {step}')
    for j in range(a[i]):
        print(f'{(2**i)-1+step*j}') # Verified expression start qubit
        print(f'{(2**(i+1))-1+step*j}') # Verified expression end qubit
    print('stop')

step = 2
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
stop
step = 4
1
3
5
7
9
11
13
15
stop
step = 8
3
7
11
15
stop
step = 16
7
15
stop
